In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import punkt
import string
from tqdm import tqdm as tq
from collections import Counter
import math
import numpy as np

In [2]:

def preprocess(text):
    ### Converting to lowercase
    text = text.lower()
    
    ### Removing Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    ### Converting strings to tokens of individual words in a complete string
    words = word_tokenize(text_p)
    
    ### Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    ### Get the root of the word i.e stemming
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return stemmed


In [3]:
def findquery(query,data,inv,voc):
    try:
        query_prep=preprocess(query)
        query_prep=[voc[i] for i in query_prep]
        doc=set(inv[query_prep[0]])
        
        try:
            for i in query_prep[1:]:
                doc=doc.intersection(inv[i])
        except:
            pass
        
        if doc==set():
            print('is not in the dataframe')
            return None

        return data[data.index.isin(doc)][['placeName','placeDesc','placeURL']]
    except:
        print('is not in the dataframe')
    

In [10]:
query='bb'
findquery(query,data,inv,voc)

,placeName,placeDesc,placeURL
785,A Christmas Story House and Museum,"For many Americans, the 1983 film A Christmas ...",https://www.atlasobscura.com/places/a-christma...
1365,Hatch Show Print,“Advertising without posters is like fishing w...,https://www.atlasobscura.com/places/hatch-show...
1439,Hicksville Trailer Palace,"On Foxy Flats Road inside Joshua Tree, artisti...",https://www.atlasobscura.com/places/hicksville...
1699,Castle Post,Looking more like an abandoned Medieval Times ...,https://www.atlasobscura.com/places/castle-post
1703,Sun Studio,Sam Phillips opened the Memphis Recording Serv...,https://www.atlasobscura.com/places/sun-studio
1782,Indiana Jones Home,"In the beginning of the film, Indiana Jones an...",https://www.atlasobscura.com/places/indiana-jo...
3600,Little Beaver Town,The infamous Red Ryder BB gun from A Christmas...,https://www.atlasobscura.com/places/little-bea...
4277,Abandoned St. Mary's Asylum,The isolated St. Mary’s Asylum was opened in 1...,https://www.atlasobscura.com/places/st-marys-a...
5590,Victoria's Last Resort,This B&B is like stepping into your own specia...,https://www.atlasobscura.com/places/victoria-s...
5743,The Old Gibson Guitar Factory,"Although the classic Gibson guitar, used by su...",https://www.atlasobscura.com/places/the-old-gi...


In [36]:
#data=pd.read_csv('final_dataset_2_rm_nan_url.csv')
#inv1=pd.read_csv('inv1.csv')
vocab=pd.read_csv('vocab.csv')
#invtf=pd.read_csv('invtf.csv')

In [37]:
#data=data.drop(['Unnamed: 0'], axis=1)
#inv1=inv1.drop(['Unnamed: 0'], axis=1)
vocab=vocab.drop(['Unnamed: 0'], axis=1)
#invtf=invtf.drop(['Unnamed: 0'], axis=1)

In [11]:
inv1['doc']=list(map(eval,inv1['doc']))
invtf['doc']=list(map(eval,invtf['doc']))
#vocab['N/n']=list(map(np.log,vocab['N/n'].values))
voc = dict(vocab[['word','ind']].values)
inv =  dict(inv1.values)
invtfvoc= dict(invtf.values)

TypeError: eval() arg 1 must be a string, bytes or code object

In [337]:
def tfidf(w,preplist,ind):
    return list((preplist.count(w))/len(preplist)*ind)[0]

In [336]:
a=[1,1,2,3,1,1]
a.count(1)

4

In [13]:
#### invtf generator
from tqdm import tqdm
import numpy as np
inv2={}
key=vocab['word']
for i,j in tqdm(zip(inv1['ind'],inv1['doc'])):
    w=key[int(i)]   
    inv2[i]={}
    for t in j:
        datas=data_des[data_des['ind']==t]['prep'].values.tolist()[0]
        inv2[i][t]=tfidf(w,datas,vocab['N/n'][i])

0it [00:00, ?it/s]


NameError: name 'data_des' is not defined

In [ ]:
#inv1=pd.DataFrame()
#inv1['ind']=inv2.keys()
#inv1['doc']=inv2.values()
#inv1.to_csv('invtf.csv')

In [252]:
mat= np.zeros((len(voc),7200))
for i in range(len(voc)):
    mat[i,list(invtfvoc[i].keys())]=list(invtfvoc[i].values())

In [340]:
def cosine(query,data,inv,vocab,voc,mat):
    query_prep=preprocess(query)
    
    query_list=[]
    query_val=[]
    for i in query_prep:
        if voc[i] not in query_list:
            query_list.append(voc[i])
            query_val.append(tfidf(i,query_prep,vocab[vocab['word']==i]['N/n']))
        
    datas = findquery(' '.join(query_prep),data,inv,voc)
    indexfind=list(datas.index)

    datas['Similarity']=[np.sum(query_val*mat[query_list,i])/(np.linalg.norm(query_val)*np.linalg.norm(mat[query_list,i])) for i in indexfind]
    
    return datas.sort_values(by=['Similarity'], ascending=False)  

In [345]:
query='american museum'
cosine(query,data,inv,vocab,voc,mat)

[1.0766038459597225, 0.7883240448055548]
1.3343652576239595


,placeName,placeDesc,placeURL,Similarity
5317,Smithsonian Sushi Collection,The American History Museum has collected an a...,https://www.atlasobscura.com/places/smithsonia...,0.999296
3961,Tamástslikt Cultural Institute,"The Tamástslikt Cultural Institute, situated o...",https://www.atlasobscura.com/places/tamastslik...,0.995444
1123,Mercer Museum and Fonthill Castle,"Henry Chapman Mercer, a renowned archaeologist...",https://www.atlasobscura.com/places/fonthill,0.995444
887,"Oak Ridge ""The Secret City""",The city of Oak Ridge was established by the U...,https://www.atlasobscura.com/places/the-secret...,0.995444
1962,Off the Rez Cafe,The U.S. government’s forced relocation of Nat...,https://www.atlasobscura.com/places/off-the-re...,0.995444
...,...,...,...,...
4968,Oscar Getz Museum of Whiskey History,If there is a place on earth that knows its wh...,https://www.atlasobscura.com/places/oscar-getz...,0.744352
1892,National World War II Museum,"Perhaps once thought too narrowly focused, thi...",https://www.atlasobscura.com/places/national-w...,0.729136
1781,National Cryptologic Museum,Update as of October 2021: Currently closed fo...,https://www.atlasobscura.com/places/national-c...,0.729136
1206,Harvard Museum of Natural History,Collecting three different institutions into o...,https://www.atlasobscura.com/places/harvard-mu...,0.716601
